In [2]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pickle

In [3]:
#Setup Selenium
drive = webdriver.Chrome(ChromeDriverManager().install())
drive.maximize_window()
wait = WebDriverWait(drive, 10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/phanvanhung/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache
/var/folders/2r/vgdn5c2j5fb567xb2nm5_qfm0000gn/T/ipykernel_63218/1106099800.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  drive = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
#Getting informantion from each house websit:
def get_info():
    house_location = drive.find_element(By.XPATH, value = '//address[@class="post-address"]').text
    house_price = drive.find_element(By.XPATH, value = '//div[@class="item price"]').text
    house_area = drive.find_element(By.XPATH, value = '//div[@class="item acreage"]').text
    house_name = drive.find_element(By.XPATH, value = '//a[@style and @href and @title]').get_attribute('title')
    house_features = drive.find_element(By.XPATH, value = '//div[@class = "section-content"]//table[@class="table"]')
    house_features = house_features.find_elements(By.XPATH, value = '//tr')
    house_info = {}
    house_info['Location'] = house_location
    house_info['Price'] = house_price
    house_info['Name'] = house_name
    house_info['Area'] = house_area
    for index in range(len(house_features)):
        house_info['Feature' + str(index)] = house_features[index].text
    return house_info

In [4]:
#Scrapping
page = 1
house_infos = list()
while True:
    check = False
    try:
        drive.get('https://phongtro123.com/tinh-thanh/da-nang?page=' + str(page))
    except: break
    try:
        drive.find_element(By.XPATH, value = '//p[@style="color: red; text-align: center;"]')
    except: pass
    else:
        check = True
    if (check == True): break
    houses = drive.find_elements(By.XPATH, value = '//h3[@class="post-title"]//a[@style and @href]')
    if (houses == []): break
    for i in range(len(houses)):
        houses[i] = houses[i].get_attribute('href')
    for house in houses:
        drive.get(house)
        try:
            house_infos.append(get_info())
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('../Raw_Data/phongtro123.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
if (house_infos):
    with open('../Raw_Data/phongtro123.pickle','wb') as handle:
        pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
drive.close()

KeyboardInterrupt: 

In [3]:
with open("../Raw_Data/phongtro123.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset.drop(columns = ['Feature7','Feature8','Feature9','Feature0'], axis = 1)
dataset.head()

,Location,Price,Name,area,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10
0,"Địa chỉ: 408/4 Tôn Đức Thắng, Phường Hòa Minh,...",1.3 triệu/tháng,"Cho thuê phòng trọ đường Tôn Đức Thắng, Liên C...",20m2,Mã tin: #602257,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin VIP 3,"Ngày đăng: Chủ Nhật, 14:10 12/06/2022","Ngày hết hạn: Thứ 6, 14:10 17/06/2022",Liên hệ: NGUYỄN HOÀNG ĐIỀN,Điện thoại: 0934050705,Zalo 0934050705,NaN
1,"Địa chỉ: Phường Nại Hiên Đông, Quận Sơn Trà, Đ...",3 triệu/tháng,Nhà biệt thự còn dư nhiều phòng cho thuê lâu d...,25m2,Mã tin: #602355,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 3, 08:31 14/06/2022","Ngày hết hạn: Thứ 6, 08:31 24/06/2022",Liên hệ: Hoàng Hương,Điện thoại: 0983293294,Zalo 0983293294,NaN
2,"Địa chỉ: k123/97/58 Cù Chính Lan, Phường Hòa K...",1.5 triệu/tháng,"Cho thuê phòng trọ (riêng chủ) gần chợ, gần tr...",15m2,Mã tin: #589224,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 5, 11:25 09/06/2022","Ngày hết hạn: Chủ Nhật, 11:25 19/06/2022",Liên hệ: Phạm Thái Bình,Điện thoại: 0905159986,Zalo 0905159986,NaN
3,"Địa chỉ: Đường Vương Thừa Vũ, Phường An Hải Bắ...",2.6 triệu/tháng,Cho thuê tại Ngã tư đường Vương Thừa Vũ + Trần...,30m2,Mã tin: #602082,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 5, 09:16 09/06/2022","Ngày hết hạn: Chủ Nhật, 09:16 19/06/2022",Liên hệ: Tăng Thùy Trâm,Điện thoại: 0935993328,Zalo 0935993328,NaN
4,"Địa chỉ: 493 Tôn Đức Thắng, Phường Hòa Khánh N...",3.2 triệu/tháng,Cho thuê phòng trọ full nội thất đẹp. Giá: từ ...,40m2,Mã tin: #602055,Khu vực Cho thuê phòng trọ Đà Nẵng,"Loại tin rao: Phòng trọ, nhà trọ",Đối tượng thuê: Tất cả,Gói tin: Tin thường,"Ngày đăng: Thứ 4, 14:35 08/06/2022","Ngày hết hạn: Thứ 7, 14:35 18/06/2022",Liên hệ: Đinh Thùy Dương,Điện thoại: 0983283515,Zalo 0983283515,NaN


In [6]:
dataset.to_csv('../Raw_Data/phongtro123.csv')